In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
from Downloads.recall_tracking import maude_data_cleaning as cleaning

In [3]:
pwd

'/Users/thanhng'

In [4]:
cd /Users/thanhng/Downloads/recall_tracking

/Users/thanhng/Downloads/recall_tracking


In [5]:
sorted_data = pd.read_csv('data_recall_1819.csv')
print(sorted_data.shape)
sorted_data.head()

(624361, 9)


,dataset,mdr_report_key,event_type,product_problems,adverse_event_flag,date_of_event,date_report,date_received,web_recall_yr
0,agilis,1626312,Malfunction,NaN,N,20100113.0,20100114.0,20100210,2018.0
1,agilis,1639306,Malfunction,NaN,N,20100216.0,20100216.0,20100315,2018.0
2,agilis,1639310,Malfunction,NaN,N,20100217.0,20100217.0,20100315,2018.0
3,agilis,1650539,Injury,NaN,Y,20100304.0,20100304.0,20100401,2018.0
4,agilis,1658874,Other,NaN,N,20100318.0,20100323.0,20100413,2018.0


In [6]:
sorted_data.dtypes

dataset                object
mdr_report_key          int64
event_type             object
product_problems       object
adverse_event_flag     object
date_of_event         float64
date_report           float64
date_received           int64
web_recall_yr         float64
dtype: object

In [7]:
pd.options.display.float_format = '{:.0f}'.format

In [8]:
sorted_data['date_of_event']= sorted_data['date_of_event'].astype('float')
sorted_data['date_of_event']

0        20100113
1        20100216
2        20100217
3        20100304
4        20100318
           ...   
624356   20181226
624357   20181226
624358   20181226
624359   20181226
624360   20181226
Name: date_of_event, Length: 624361, dtype: float64

### Count number of reports for each device in maude

In [9]:
tracking_df = cleaning.CREATE_TRACKING_DF(sorted_data)
tracking_df

,dataset,counts_in_maude
0,acuity,507
1,agilis,544
2,airlife,113
3,alaris,166
4,alpha_omega,4
...,...,...
78,verio,80044
79,vial2bag,29
80,vitek,132
81,vns,38


In [10]:
#take a random dataset and check if count is correct
group_dataset = sorted_data.groupby('dataset')
df_index = sorted_data.loc[group_dataset.groups.get('agilis')].index
print(sorted_data.loc[df_index].shape)
sorted_data.loc[df_index].isna().sum()

(544, 9)


dataset                 0
mdr_report_key          0
event_type              0
product_problems      158
adverse_event_flag     10
date_of_event          38
date_report             0
date_received           0
web_recall_yr           0
dtype: int64

### Count missing in AE, event_date and event_type

In [11]:
#count missing in event_type
tracking_df['missing_in_AE'] = 'none'
tracking_df['missing_event_date'] = 'none'
tracking_df['missing_event_type'] = 'none'
tracking_df = cleaning.COUNT_MISSING(sorted_data, tracking_df)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type
0,acuity,507,0,0,none
1,agilis,544,10,38,none
2,airlife,113,26,5,none
3,alaris,166,1,28,none
4,alpha_omega,4,0,0,none
...,...,...,...,...,...
78,verio,80044,8,0,none
79,vial2bag,29,3,0,none
80,vitek,132,2,95,none
81,vns,38,3,0,none


### COUNT # of deaths, malfunction, injury

In [12]:
### COUNT # of deaths, malfunction, injury
tracking_df['#_of_malfunction'] = 0
tracking_df['#_of_injury'] = 0
tracking_df['#_of_death'] = 0

tracking_df = cleaning.COUNT_EVENT_TYPE(sorted_data, tracking_df)
# tracking_df['missing_event_type'] = tracking_df['counts_in_maude'] \
#                                 - tracking_df[['#_of_malfunction','#_of_injury', '#_of_death']].sum(axis = 1)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death
0,acuity,507,0,0,none,505,1,0
1,agilis,544,10,38,none,219,302,19
2,airlife,113,26,5,none,63,35,0
3,alaris,166,1,28,none,136,29,0
4,alpha_omega,4,0,0,none,1,3,0
...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,none,78509,1350,0
79,vial2bag,29,3,0,none,11,17,0
80,vitek,132,2,95,none,120,9,3
81,vns,38,3,0,none,22,14,1


In [13]:
#take a random dataset and check if function performs correctly
group_dataset = sorted_data.groupby('dataset')
df_index = sorted_data.loc[group_dataset.groups.get('alpha_omega')].index
sorted_data.loc[df_index]

,dataset,mdr_report_key,event_type,product_problems,adverse_event_flag,date_of_event,date_report,date_received,web_recall_yr
52737,alpha_omega,6683773,Malfunction,NaN,N,20161130,20170106,20170703,2019
52738,alpha_omega,6685303,Injury,['Incorrect Device Or Component Shipped'],Y,20170601,20170630,20170704,2019
52739,alpha_omega,7306330,Injury,NaN,Y,20180131,20180228,20180301,2019
52740,alpha_omega,7780725,Injury,NaN,Y,20180724,20180807,20180814,2019


In [14]:

tracking_df['missing_event_type'] = tracking_df['counts_in_maude'] \
                                - tracking_df[['#_of_malfunction','#_of_injury', '#_of_death']].sum(axis = 1)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death
0,acuity,507,0,0,1,505,1,0
1,agilis,544,10,38,4,219,302,19
2,airlife,113,26,5,15,63,35,0
3,alaris,166,1,28,1,136,29,0
4,alpha_omega,4,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0
79,vial2bag,29,3,0,1,11,17,0
80,vitek,132,2,95,0,120,9,3
81,vns,38,3,0,1,22,14,1


In [15]:
##count AE
tracking_df['#_of_AE'] = 'none'
group_dataset = sorted_data.groupby('dataset')
tracking_df = cleaning.COUNT_AE(sorted_data, tracking_df)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE
0,acuity,507,0,0,1,505,1,0,5
1,agilis,544,10,38,4,219,302,19,321
2,airlife,113,26,5,15,63,35,0,34
3,alaris,166,1,28,1,136,29,0,19
4,alpha_omega,4,0,0,0,1,3,0,3
...,...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0,1362
79,vial2bag,29,3,0,1,11,17,0,17
80,vitek,132,2,95,0,120,9,3,16
81,vns,38,3,0,1,22,14,1,14


In [16]:
####get the first malf, injury, death
tracking_df['first_malfunction'] = 0
tracking_df['first_injury'] = 0
tracking_df['first_death'] = 0

# tracking_df

tracking_df = cleaning.GET_FIRST_EVENT_TYPE(sorted_data, tracking_df)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death
0,acuity,507,0,0,1,505,1,0,5,20080326,20110309,0
1,agilis,544,10,38,4,219,302,19,321,20100113,20100304,20131114
2,airlife,113,26,5,15,63,35,0,34,20100315,20100831,0
3,alaris,166,1,28,1,136,29,0,19,20100120,20111110,0
4,alpha_omega,4,0,0,0,1,3,0,3,20161130,20170601,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0,1362,20111031,20130701,0
79,vial2bag,29,3,0,1,11,17,0,17,20150909,20181030,0
80,vitek,132,2,95,0,120,9,3,16,20100921,20100201,20110812
81,vns,38,3,0,1,22,14,1,14,20100420,20100901,20181018


In [17]:
#MAUDE FIRST DATE
tracking_df = cleaning.GET_FIRST_DATE(sorted_data, tracking_df)
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date
0,acuity,507,0,0,1,505,1,0,5,20080326,20110309,0,20080326
1,agilis,544,10,38,4,219,302,19,321,20100113,20100304,20131114,20100113
2,airlife,113,26,5,15,63,35,0,34,20100315,20100831,0,20100315
3,alaris,166,1,28,1,136,29,0,19,20100120,20111110,0,20100120
4,alpha_omega,4,0,0,0,1,3,0,3,20161130,20170601,0,20161130
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0,1362,20111031,20130701,0,20111031
79,vial2bag,29,3,0,1,11,17,0,17,20150909,20181030,0,20150909
80,vitek,132,2,95,0,120,9,3,16,20100921,20100201,20110812,20100201
81,vns,38,3,0,1,22,14,1,14,20100420,20100901,20181018,20100420


In [18]:
#take a random dataset and check if function performs correctly
group_dataset = sorted_data.groupby('dataset')
df_index = sorted_data.loc[group_dataset.groups.get('alpha_omega')].index
sorted_data.loc[df_index]

,dataset,mdr_report_key,event_type,product_problems,adverse_event_flag,date_of_event,date_report,date_received,web_recall_yr
52737,alpha_omega,6683773,Malfunction,NaN,N,20161130,20170106,20170703,2019
52738,alpha_omega,6685303,Injury,['Incorrect Device Or Component Shipped'],Y,20170601,20170630,20170704,2019
52739,alpha_omega,7306330,Injury,NaN,Y,20180131,20180228,20180301,2019
52740,alpha_omega,7780725,Injury,NaN,Y,20180724,20180807,20180814,2019


### take date different

In [19]:
def CONVERT_COLUMN_TO_DATETIME(data, column):
    data[column] = pd.to_datetime(data[column], format='%Y%m%d', errors='coerce')
    return data[column]

In [20]:
sorted_data['date_of_event_edit'] = CONVERT_COLUMN_TO_DATETIME(sorted_data, 'date_of_event')
sorted_data['date_of_event_edit']

0        2010-01-13
1        2010-02-16
2        2010-02-17
3        2010-03-04
4        2010-03-18
            ...    
624356   2018-12-26
624357   2018-12-26
624358   2018-12-26
624359   2018-12-26
624360   2018-12-26
Name: date_of_event_edit, Length: 624361, dtype: datetime64[ns]

In [21]:
def REMOVE_MALFUNCTION(data):
    #extract data
    inj=data[data['event_type']!='Malfunction']
    #print results
    inj_MEAN = inj['date_of_event_edit'].diff().mean().days
    inj_MEDIAN = inj['date_of_event_edit'].diff().median().days
    return inj_MEAN, inj_MEDIAN

In [22]:
sorted_data['date_diff'] = '1900-01-01'

In [23]:
dataset_name = tracking_df['dataset']
group_tracking = tracking_df.groupby('dataset')
for name in dataset_name:
    df_index = sorted_data.loc[group_dataset.groups.get(name)].index
    sorted_data.loc[df_index, 'date_diff'] = sorted_data.loc[df_index, 'date_of_event_edit'].diff().dt.days
    diff_col = sorted_data.loc[df_index, 'date_diff']
  #  print(sorted_data.loc[df_index].filter(['date_of_event_edit','date_diff']))
    diff_MEAN = diff_col.mean()
    diff_MEDIAN = diff_col.median()
   
    tracking_df.loc[group_tracking.groups.get(name),'mean_date_diff_bw_event'] = np.round(diff_MEAN)
    tracking_df.loc[group_tracking.groups.get(name), 'median_date_diff_bw_event'] = np.round(diff_MEDIAN)

    df = sorted_data.loc[df_index]
    inj_MEAN, inj_MEDIAN = REMOVE_MALFUNCTION(df)

    tracking_df.loc[group_tracking.groups.get(name),'mean_date_diff_bw_deathinj'] = inj_MEAN
    tracking_df.loc[group_tracking.groups.get(name), 'median_date_diff_bw_deathinj'] = inj_MEDIAN


/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1112: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [24]:
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date,mean_date_diff_bw_event,median_date_diff_bw_event,mean_date_diff_bw_deathinj,median_date_diff_bw_deathinj
0,acuity,507,0,0,1,505,1,0,5,20080326,20110309,0,20080326,7,3,338,338
1,agilis,544,10,38,4,219,302,19,321,20100113,20100304,20131114,20100113,7,3,11,6
2,airlife,113,26,5,15,63,35,0,34,20100315,20100831,0,20100315,33,18,69,43
3,alaris,166,1,28,1,136,29,0,19,20100120,20111110,0,20100120,26,9,108,72
4,alpha_omega,4,0,0,0,1,3,0,3,20161130,20170601,0,20161130,200,183,209,209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0,1362,20111031,20130701,0,20111031,0,0,1,1
79,vial2bag,29,3,0,1,11,17,0,17,20150909,20181030,0,20150909,43,1,3,0
80,vitek,132,2,95,0,120,9,3,16,20100921,20100201,20110812,20100201,79,34,261,212
81,vns,38,3,0,1,22,14,1,14,20100420,20100901,20181018,20100420,92,70,217,142


In [25]:
#take a random dataset and check if calculate correct
df_index = sorted_data.loc[group_dataset.groups.get('ventstar')].index
sorted_data.loc[df_index, 'date_diff'] = sorted_data.loc[df_index, 'date_of_event_edit'].diff().dt.days
diff_col = sorted_data.loc[df_index, 'date_diff']
print(sorted_data.loc[df_index].filter(['date_of_event_edit','date_diff']))

      date_of_event_edit date_diff
52713                NaT       NaN
52714                NaT       NaN
52715                NaT       NaN
52716                NaT       NaN
52717                NaT       NaN
52718         2010-03-26       NaN
52719         2014-03-07      1442
52720         2015-01-19       318
52721         2015-07-10       172
52722         2016-07-26       382
52723         2016-07-26         0
52724         2016-07-26         0
52725         2016-09-15        51
52726         2016-11-25        71
52727         2017-08-23       271
52728         2017-12-03       102
52729         2017-12-03         0
52730         2017-12-06         3
52731         2018-10-23       321
52732         2019-01-04        73
52733         2019-01-04         0
52734         2019-04-08        94
52735         2019-05-16        38
52736         2019-08-28       104


In [26]:
check_df = sorted_data.loc[df_index].filter(['date_of_event_edit','date_diff'])
print('MEAN: ', check_df['date_diff'].mean())
print('MEDIAN: ', check_df['date_diff'].median())

MEAN:  191.22222222222223
MEDIAN:  83.5


In [27]:
tracking_df[tracking_df['dataset'] == 'ventstar']

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date,mean_date_diff_bw_event,median_date_diff_bw_event,mean_date_diff_bw_deathinj,median_date_diff_bw_deathinj
77,ventstar,24,0,5,0,18,6,0,6,20140307,20100326,0,20100326,191,84,801,637


### GET # of events from 1st event to 1st AE

In [28]:
tracking_df['#_events_from_1st_event_to_1st_AE'] = 'none'

count =0
for (name, index) in zip(dataset_name, range(tracking_df.shape[0])):

   df = sorted_data.loc[group_dataset.groups.get(name)].reset_index()
   AE_first = df['adverse_event_flag'][0]
   if AE_first == 'Y':
       tracking_df.loc[index, '#_events_from_1st_event_to_1st_AE'] = 0
   if AE_first == 'N':
        index_df = pd.DataFrame(df.groupby(['adverse_event_flag']).first()['index'])
        index_trans = index_df.transpose()
        if len(index_trans.columns) == 1:
           tracking_df.loc[index, '#_events_from_1st_event_to_1st_AE'] = 0
        if len(index_trans.columns) > 1:
            res = index_trans['Y'].values - index_trans['N'].values
            tracking_df.loc[index,  '#_events_from_1st_event_to_1st_AE'] = res
tracking_df

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date,mean_date_diff_bw_event,median_date_diff_bw_event,mean_date_diff_bw_deathinj,median_date_diff_bw_deathinj,#_events_from_1st_event_to_1st_AE
0,acuity,507,0,0,1,505,1,0,5,20080326,20110309,0,20080326,7,3,338,338,33
1,agilis,544,10,38,4,219,302,19,321,20100113,20100304,20131114,20100113,7,3,11,6,3
2,airlife,113,26,5,15,63,35,0,34,20100315,20100831,0,20100315,33,18,69,43,14
3,alaris,166,1,28,1,136,29,0,19,20100120,20111110,0,20100120,26,9,108,72,0
4,alpha_omega,4,0,0,0,1,3,0,3,20161130,20170601,0,20161130,200,183,209,209,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,verio,80044,8,0,185,78509,1350,0,1362,20111031,20130701,0,20111031,0,0,1,1,0
79,vial2bag,29,3,0,1,11,17,0,17,20150909,20181030,0,20150909,43,1,3,0,11
80,vitek,132,2,95,0,120,9,3,16,20100921,20100201,20110812,20100201,79,34,261,212,0
81,vns,38,3,0,1,22,14,1,14,20100420,20100901,20181018,20100420,92,70,217,142,2


In [29]:
#take a random dataset and check if calculate correct
df_index = sorted_data.loc[group_dataset.groups.get('vial2bag')].index
sorted_data.loc[df_index, ['dataset', 'event_type', 'adverse_event_flag']]


,dataset,event_type,adverse_event_flag
624332,vial2bag,Malfunction,N
624333,vial2bag,Malfunction,N
624334,vial2bag,Malfunction,N
624335,vial2bag,Malfunction,N
624336,vial2bag,Malfunction,NaN
624337,vial2bag,Malfunction,NaN
624338,vial2bag,Malfunction,N
624339,vial2bag,Malfunction,N
624340,vial2bag,Malfunction,N
624341,vial2bag,Malfunction,NaN


### extract only dates columns in tracking_df

In [30]:
data_1 = pd.read_csv('tracking_1819.csv')
data_1 = data_1.filter(['dataset','recall_date'])
tracking_date = tracking_df.filter(['dataset', 'first_malfunction', 'first_injury', 'first_death','maude_first_date'])
data_dates = pd.merge(tracking_date, data_1, on='dataset')
data_dates

,dataset,first_malfunction,first_injury,first_death,maude_first_date,recall_date
0,acuity,20080326,20110309,0,20080326,20040213
1,agilis,20100113,20100304,20131114,20100113,20180102
2,airlife,20100315,20100831,0,20100315,20180810
3,alaris,20100120,20111110,0,20100120,20190718
4,alpha_omega,20161130,20170601,0,20161130,20190516
...,...,...,...,...,...,...
76,verio,20111031,20130701,0,20111031,20120402
77,vial2bag,20150909,20181030,0,20150909,20190128
78,vitek,20100921,20100201,20110812,20100201,20180823
79,vns,20100420,20100901,20181018,20100420,20191220


In [31]:
column_convert = ['recall_date', 'first_malfunction', 'first_injury', 'first_death']
#convert column to datetime object

def CONVERT_COLUMN_TO_DATETIME(data, column):
    data[column] = pd.to_datetime(data[column], format='%Y%m%d', errors='coerce')
    return data[column]

for col in column_convert:
    data_dates[col] = CONVERT_COLUMN_TO_DATETIME(data_dates, col)
data_dates

,dataset,first_malfunction,first_injury,first_death,maude_first_date,recall_date
0,acuity,2008-03-26,2011-03-09,NaT,20080326,2004-02-13
1,agilis,2010-01-13,2010-03-04,2013-11-14,20100113,2018-01-02
2,airlife,2010-03-15,2010-08-31,NaT,20100315,2018-08-10
3,alaris,2010-01-20,2011-11-10,NaT,20100120,2019-07-18
4,alpha_omega,2016-11-30,2017-06-01,NaT,20161130,2019-05-16
...,...,...,...,...,...,...
76,verio,2011-10-31,2013-07-01,NaT,20111031,2012-04-02
77,vial2bag,2015-09-09,2018-10-30,NaT,20150909,2019-01-28
78,vitek,2010-09-21,2010-02-01,2011-08-12,20100201,2018-08-23
79,vns,2010-04-20,2010-09-01,2018-10-18,20100420,2019-12-20


In [32]:
column_different = ['recall-malf', 'recall-injury', 'recall-death']
column_being_subtracted = column_convert[1:]


def DATE_DIFFERENT_IN_YEARS(data, col1, col2, col_result):
    data[col_result] = (data[col1] - data[col2]).dt.days
    return data[col_result]


for (col_res,col_subtract)  in zip(column_different, column_being_subtracted):
    data_dates[col_res] =  DATE_DIFFERENT_IN_YEARS(data_dates, 'recall_date', col_subtract, col_res)/365

data_dates

,dataset,first_malfunction,first_injury,first_death,maude_first_date,recall_date,recall-malf,recall-injury,recall-death
0,acuity,2008-03-26,2011-03-09,NaT,20080326,2004-02-13,-4,-7,nan
1,agilis,2010-01-13,2010-03-04,2013-11-14,20100113,2018-01-02,8,8,4
2,airlife,2010-03-15,2010-08-31,NaT,20100315,2018-08-10,8,8,nan
3,alaris,2010-01-20,2011-11-10,NaT,20100120,2019-07-18,9,8,nan
4,alpha_omega,2016-11-30,2017-06-01,NaT,20161130,2019-05-16,2,2,nan
...,...,...,...,...,...,...,...,...,...
76,verio,2011-10-31,2013-07-01,NaT,20111031,2012-04-02,0,-1,nan
77,vial2bag,2015-09-09,2018-10-30,NaT,20150909,2019-01-28,3,0,nan
78,vitek,2010-09-21,2010-02-01,2011-08-12,20100201,2018-08-23,8,9,7
79,vns,2010-04-20,2010-09-01,2018-10-18,20100420,2019-12-20,10,9,1


In [33]:
data_dates.filter(column_different).describe()

,recall-malf,recall-injury,recall-death
count,78,76,52
mean,8,7,6
std,8,6,5
min,-6,-7,-11
25%,5,3,2
50%,9,8,6
75%,10,10,9
max,59,27,18


In [34]:
''' On average, it takes 6 years from first death event to recall date, \
7 years from first injury for device to be recalled, \
8 years from first malfunction to recall date''' 

' On average, it takes 6 years from first death event to recall date, 7 years from first injury for device to be recalled, 8 years from first malfunction to recall date'

In [35]:
def GET_THE_MAX_ROW(data, col):
    return data.loc[data[col].idxmax()]

GET_THE_MAX_ROW(data_dates, 'recall-malf')

dataset                           unicel
first_malfunction    1960-10-22 00:00:00
first_injury         2007-03-04 00:00:00
first_death          2012-02-01 00:00:00
maude_first_date                19601022
recall_date          2019-05-23 00:00:00
recall-malf                           59
recall-injury                         12
recall-death                           7
Name: 72, dtype: object

In [36]:
tracking_df[tracking_df['dataset'] == 'unicel']

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date,mean_date_diff_bw_event,median_date_diff_bw_event,mean_date_diff_bw_deathinj,median_date_diff_bw_deathinj,#_events_from_1st_event_to_1st_AE
74,unicel,18469,113,47,199,17586,683,1,799,19601022,20070304,20120201,19601022,1,0,7,1,65


In [37]:
def GET_THE_MIN_ROW(data, col):
    return data.loc[data[col].idxmin()]

GET_THE_MIN_ROW(data_dates, 'recall-death')

dataset                          marquis
first_malfunction    2010-05-14 00:00:00
first_injury         2010-02-09 00:00:00
first_death          2016-01-07 00:00:00
maude_first_date                20100209
recall_date          2005-03-11 00:00:00
recall-malf                           -5
recall-injury                         -5
recall-death                         -11
Name: 43, dtype: object

In [38]:
tracking_df[tracking_df['dataset'] == 'marquis']

,dataset,counts_in_maude,missing_in_AE,missing_event_date,missing_event_type,#_of_malfunction,#_of_injury,#_of_death,#_of_AE,first_malfunction,first_injury,first_death,maude_first_date,mean_date_diff_bw_event,median_date_diff_bw_event,mean_date_diff_bw_deathinj,median_date_diff_bw_deathinj,#_events_from_1st_event_to_1st_AE
45,marquis,188,0,0,0,26,159,3,136,20100514,20100209,20160107,20100209,15,0,17,0,8
